In [ ]:
import torch
import torchvision
import os
import numpy as np
import matplotlib.patches as patches
import matplotlib.pyplot as plt
import torchvision
from torchvision import transforms, datasets, models
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
import time
import random
import shutil
import os
import glob
import pandas as pd
import xml.etree.ElementTree as ET
import cv2
from PIL import Image as im
import albumentations

# equation from https://pillow.readthedocs.io/en/stable/reference/Image.html#PIL.Image.Image.convert
def togray(rgb):
    return np.dot(rgb[...,:3], [0.2989, 0.5870, 0.1140])

In [ ]:
torch.cuda.empty_cache() 

In [ ]:
%cd Faster_RCNN

In [ ]:
img_lst = [
# I remove some images during 11 rounds of image quality check
'Gqm36jq0tQuYnLqWzGo3Ug','Zs_xFrbRC5Q156zWzpU2Qg','M2eB1wtGEsLtiOzTIpCALA','mI6xbDdBbBZ_yBqKSJB30g',
'mhFpDqcxuckfJho3xaJWRw','JqFDjRUDWjOxPlag5cDViQ','bSScQ4PbF49TOpQ3IIhzrQ',
'mPxV4HxCk-85SNCVdUeKdw','X8Mw7DFG24oN1uk91VQheQ','dtGJg_hFK_2G-IyqjxAutA','lMU6tm4TwMM7NYCKQ15Bgg',
'Sh0b5UAEOzGJDgDkvdPBqw','izNoQd5-yz_nydU5jRNwfQ','32e2-fiOfRGH-aOi_FhGwA','7FNvh4Ya_3aIYDCSVMUKNQ',
'online_50','online3','online_41','online_33','online74','online68','online65','online77',
'online72','online64','online_42','online_35','online63','online_32',
'online_40','online53','online13','online19','online75','online51',
'online5','online_25','online_36','online61',
'online57','online55','online20','online_26','online_34','online62','online1','online59','online4',
'online14','online56','online66','online_37','online69','online76','online15','online12',
'online_45','online60','online67','online_27','online58','online7','online54','online_31',
'online_39','online73','online22','online_49','online_46','online70','online_24','online6',
'online21','online11','online_44','online_43','online16',
'online71','online_47','online_30','online_38','online52','online_48','online23',
# 'GOPR6632',# 'GOPR6638',# 'GOPR6657',# 'GOPR6707',# 'GOPR6797', # 'YykKYLkLL6IX_jiizQf9Lg',# 'ty8tjWSRCzPHH5CUUPXppQ',
# 'online17', # 'online10',# 'online9',# 'online18', # 'online8',# 'online_28',# 'online2',
'0027','0029','0101','6585','6587','6618','6622','6623','6624','6625','6645','6647','6690','6739','6741','6676',
'6673','6679','6678','6686','6682','6729','6691','6755','6730']

In [ ]:
source = '/home/yuchen/venv/Faster_RCNN/data_sign/total_images'
train_image_source = '/home/yuchen/venv/Faster_RCNN/data_sign/train_images'
train_label_source = '/home/yuchen/venv/Faster_RCNN/data_sign/train_labels'
label_source = '/home/yuchen/venv/Faster_RCNN/data_sign/total_labels'
valid_image_dest = '/home/yuchen/venv/Faster_RCNN/data_sign/valid_images'
valid_label_dest = '/home/yuchen/venv/Faster_RCNN/data_sign/valid_labels'
test_image_source = '/home/yuchen/venv/Faster_RCNN/data_sign/test_images'
test_label_source = '/home/yuchen/venv/Faster_RCNN/data_sign/test_labels'

files = glob.glob(train_image_source + '/*')
files.extend(glob.glob(train_label_source + '/*'))
files.extend(glob.glob(valid_image_dest + '/*'))
files.extend(glob.glob(valid_label_dest + '/*'))

# remove existing files
for f in files:
    os.remove(f)

# copy everything to train set
for file_name in glob.glob('/home/yuchen/venv/Faster_RCNN/data_sign/total_images/*'):
    file_name_temp = file_name[53:-4]
    if file_name_temp in img_lst:
        shutil.copy(os.path.join(source, file_name), train_image_source)
        shutil.copy(os.path.join(label_source, label_source + '/' + file_name_temp +'.xml'), train_label_source)
    
# 8/2 train/valid split
files = os.listdir(train_image_source)
no_of_files = len(files) // 5

random.seed(42)
for file_name in random.sample(files, no_of_files):
    shutil.move(os.path.join(train_image_source, file_name), valid_image_dest)
    shutil.move(os.path.join(train_label_source, file_name[:-4]+'.xml'), valid_label_dest)

In [ ]:
files = glob.glob('/home/yuchen/venv/Faster_RCNN/data_sign/train_labels/*')
files.extend(glob.glob('/home/yuchen/venv/Faster_RCNN/data_sign/valid_labels/*'))

for file_name in files:
    mytree = ET.parse(file_name)
    myroot = mytree.getroot()
    img_w = int(myroot[4].find('width').text)
    img_h = int(myroot[4].find('height').text)

    lst = []
    ct = 0
    for obj in myroot.iter('object'):
        ct += 1
        xmin = int(obj.find('bndbox').find('xmin').text)
        xmax = int(obj.find('bndbox').find('xmax').text)
        ymin = int(obj.find('bndbox').find('ymin').text)
        ymax = int(obj.find('bndbox').find('ymax').text)

        w = (xmax-xmin) / img_w * 1280
        h = (ymax-ymin) / img_h * 1280
        
        if w*h <= 300:
            lst.append(obj)
            
    if len(lst) != 0:
        if ct == len(lst):
            os.remove(file_name)
            os.remove(glob.glob('/home/yuchen/venv/Faster_RCNN/data_sign/' + file_name[40:45] + '_images/' + file_name[52:-4] + '*')[0])
            print('remove the entire file ' + file_name)
        else:
            for i in reversed(lst):
                myroot.remove(i)
            mytree.write(file_name)
            print('removed ' + str(len(lst)) + ' bounding boxes from ' + file_name[53:])

In [ ]:
# replace each image's 3rd channel with its grayscale version
# this will lead images to appear more red and green and less blue, 
# and can improve the model's performance on both validation and test sets
files = glob.glob('/home/yuchen/venv/Faster_RCNN/data_sign/train_images/*')
files.extend(glob.glob('/home/yuchen/venv/Faster_RCNN/data_sign/valid_images/*'))

for file_name in files:
    img = np.array(im.open(file_name).copy())
    img_copy  = togray(img)
    img[:,:,-1] = img_copy
    img = im.fromarray(img).copy()
    img.save(file_name)

In [ ]:
! python train.py \
--batch 10 \
--epochs 30 \
--name test_run